In [2]:
import pandas as pd
import numpy as np

In [3]:
#入学数据
rx_features=pd.read_csv('rx_data/rx_feature_ff.csv')
#就业数据
jy_features=pd.read_csv('jy_data/jy_feature.csv')
#心理预警数据
psy_warning_features=pd.read_csv('psy_data/psy.csv')
#学业预警
xy_warning_features=pd.read_csv('student_warning_data/student_warning_feature.csv')
#贫困生数据
poor_student_features=pd.read_csv('poor_student_data/poor_students.csv')

In [4]:
rx_features.head()

,sno,zf,tdf,female,male,ncyj,ncwj,czyj,czwj,ks_is_jf,...,ebfsx_lk,ssmzlqf_lk,hzlqf_lk,zfdj,zf_yb_d,zf_lqf_d,zf_pjf_d,zybm,xybm,zy_fs_pm
0,2012210001,533,533,1,0,0,0,0,1,1,...,476,487,487,2,-6.0,11,15,4,8,71.0
1,2012210002,484,494,1,0,0,0,1,0,0,...,390,437,458,3,15.0,47,-34,4,8,11.0
2,2012210003,520,520,1,0,0,0,1,0,1,...,412,420,415,1,-9.0,19,2,4,8,38.0
3,2012210004,578,578,1,0,0,0,0,1,1,...,478,496,539,1,1.0,0,60,4,8,215.0
4,2012210005,566,566,1,0,0,0,0,1,1,...,486,487,531,3,-4.0,1,48,4,8,161.0


In [23]:
jy_features.head()

,xh,jyzk
0,2012213477,1
1,2012210001,1
2,2012215888,1
3,2012210003,0
4,2012210257,1


In [5]:
psy_warning_features.head()

,sno,is_psy_warning
0,2012210271,1
1,2012210382,1
2,2012210453,1
3,2012210508,1
4,2012210791,1


In [9]:
#删除重复的数据
psy_warning_features=psy_warning_features.drop_duplicates()

In [10]:
xy_warning_features.head()

,sno,student_warning_lvl
0,2012211580,3
1,2012211580,3
2,2012212561,2
3,2012212561,2
4,2012212865,4


In [26]:
#删除重复的数据
xy_warning_features=xy_warning_features.drop_duplicates()

In [11]:
poor_student_features.head()

,sno,poor_levlel,is_poor
0,2012104676,2,1
1,201210857,1,1
2,2012113080,1,1
3,2012113173,2,1
4,201211393,1,1


In [13]:
#找出有错误的学号

def sno(df):
    a=[]
    for i in range(len(df)):
        if len(str(df.ix[i]))!=10:
            a.append(i)
    return a
b=sno(poor_student_features['sno'])

#有错的学号

poor_student_features['sno'].ix[b]

#修正有错误的学号

c=[2012210869,2012211394,2012210031,2012210884,2012210891,2012211082,2012211359
   ,2012211516,2012211520,2012211523,2012211574,2012212974,2012214115,2012214203]
poor_student_features['sno'].ix[b]=c

In [15]:
#构造标签数据

jy_features.rename(columns={'xh':'sno'},inplace=True)
all_features=pd.merge(jy_features,xy_warning_features,how='outer',on='sno')
all_features=pd.merge(all_features,psy_warning_features,how='outer',on='sno')

#特征数据
rx_features_f=pd.merge(rx_features,poor_student_features,how='left',on='sno')

In [16]:
#删除无用的列(只留下学号及对应的标签)

all_features=all_features.fillna(0)
all_features1=all_features.drop('student_warning_lvl',axis=1)

In [17]:
#填充缺失值

rx_features_f['poor_levlel']=rx_features_f['poor_levlel'].fillna(4)
rx_features_f['is_poor']=rx_features_f['is_poor'].fillna(0)

#连接特征与标签数据
all_features_f=pd.merge(rx_features_f,all_features1,how='left',on='sno')
#删除重复的数据
all_features_f=all_features_f.drop_duplicates()
#填充空值为0
all_features_f['jyzk']=all_features_f['jyzk'].fillna(0)

In [18]:
all_features_f.head()

,sno,zf,tdf,female,male,ncyj,ncwj,czyj,czwj,ks_is_jf,...,zf_yb_d,zf_lqf_d,zf_pjf_d,zybm,xybm,zy_fs_pm,poor_levlel,is_poor,jyzk,is_psy_warning
0,2012210001,533,533,1,0,0,0,0,1,1,...,-6.0,11,15,4,8,71.0,4.0,0.0,1.0,0.0
1,2012210002,484,494,1,0,0,0,1,0,0,...,15.0,47,-34,4,8,11.0,4.0,0.0,1.0,0.0
2,2012210003,520,520,1,0,0,0,1,0,1,...,-9.0,19,2,4,8,38.0,4.0,0.0,0.0,0.0
3,2012210004,578,578,1,0,0,0,0,1,1,...,1.0,0,60,4,8,215.0,4.0,0.0,1.0,0.0
4,2012210005,566,566,1,0,0,0,0,1,1,...,-4.0,1,48,4,8,161.0,1.0,1.0,1.0,0.0


In [19]:
#保存数据
all_features_f.to_csv('all_features/all_features.csv',index=False)